In [1]:
%%capture
import os

!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

In [2]:
import os
import pandas as pd
import numpy as np

train = pd.read_csv(os.path.join('/content/train.csv'))
test = pd.read_csv(os.path.join('/content/test.csv'))
sample_test = pd.read_csv(os.path.join('/content/sample_test.csv'))
sample_test_out = pd.read_csv(os.path.join('/content/sample_test_out.csv'))



In [3]:
import re
import os
import pandas as pd
import multiprocessing
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib

def download_image(image_link, savefolder):
    if(isinstance(image_link, str)):
        filename = Path(image_link).name
        image_save_path = os.path.join(savefolder, filename)
        if(not os.path.exists(image_save_path)):
            try:
                urllib.request.urlretrieve(image_link, image_save_path)
            except Exception as ex:
                print('Warning: Not able to download - {}\n{}'.format(image_link, ex))
        else:
            return
    return

def download_images(image_links, download_folder):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)
    results = []
    download_image_partial = partial(download_image, savefolder=download_folder)
    with multiprocessing.Pool(100) as pool:
        for result in tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)):
            results.append(result)
        pool.close()
        pool.join()


In [4]:
download_images(train.head(1000)['image_link'], '/balaji/train/')

100%|██████████| 1000/1000 [00:00<00:00, 26965.21it/s]


In [5]:
from PIL import Image

instruction = f"""
You are a helpful assistant.
The user provides a product catalog and an image. Your task is to predict the price in USD.

Important:
- Do NOT use numbers from weights, volumes, or quantities (like '12.7 Ounce', '11.25 oz') as the price.
- Consider the type of product, brand, and image cues to estimate the correct price.
- Output ONLY the price as a number in USD.
- Example: 4.89
"""

def convert_to_conversation(sample):
    image_filename = os.path.basename(sample["image_link"])
    image_path = f"/balaji/train/{image_filename}"
    image = Image.open(image_path).convert("RGB")

    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"{instruction}\n\n{sample['catalog_content']}"},
                {"type": "image", "image": image}
            ]
        },
        {
            "role": "assistant",
            "content": [
                {"type": "text", "text": str(sample["price"])}
            ]
        }
    ]

    return {"messages": conversation}

In [6]:
train = train.head(1000)
train

,sample_id,catalog_content,image_link,price
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49
...,...,...,...,...
995,116820,Item Name: Charleston Coffee Roasters | Specia...,https://m.media-amazon.com/images/I/815IfcKqrf...,12.99
996,56693,Item Name: Bosco & Roxy Christmas Packaged Pea...,https://m.media-amazon.com/images/I/71ZYl9gWvH...,16.99
997,155128,Item Name: Spicely Organics Mustard Powder (Ye...,https://m.media-amazon.com/images/I/41LiARHOI1...,35.00
998,46146,Item Name: HUNTS STEWED TOMATOES 14.5oz 3pack\...,https://m.media-amazon.com/images/I/31tUmzhkA6...,1.39


In [15]:
from PIL import Image

instruction = f"""
You are a helpful assistant.
The user provides a product catalog and an image. Your task is to predict the price in USD.

Important:
- Do NOT use numbers from weights, volumes, or quantities (like '12.7 Ounce', '11.25 oz') as the price.
- Consider the type of product, brand, and image cues to estimate the correct price.
- Output ONLY the price as a number in USD.
- Example: 4.89
"""

def convert_to_conversation(sample):
    image_filename = os.path.basename(sample["image_link"])
    image_path = f"/balaji/train/{image_filename}" # Corrected path
    image = Image.open(image_path).convert("RGB")

    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"{instruction}\n\n{sample['catalog_content']}"},
                {"type": "image", "image": image}
            ]
        },
        {
            "role": "assistant",
            "content": [
                {"type": "text", "text": str(sample["price"])}
            ]
        }
    ]

    return {"messages": conversation}

In [12]:
train.loc[0, "catalog_content"]


'Item Name: La Victoria Green Taco Sauce Mild, 12 Ounce (Pack of 6)\nValue: 72.0\nUnit: Fl Oz\n'

In [16]:
converted_dataset = []
for row in train.itertuples(index=False):
    sample = {
        "sample_id": row.sample_id,
        "catalog_content": row.catalog_content,
        "image_link": row.image_link,
        "price": row.price
    }
    converted_dataset.append(convert_to_conversation(sample))

In [17]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2.5-VL-3B-Instruct-bnb-4bit",
    load_in_4bit=True,
    use_gradient_checkpointing="unsloth",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.1: Fast Qwen2_5_Vl patching. Transformers: 4.55.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


...

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json: 0.00B [00:00, ?B/s]

In [18]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers=True,
    finetune_language_layers=True,
    finetune_attention_modules=True,
    finetune_mlp_modules=True,
    r=32,  # Increased for better learning, aiming SMAPE 20-40
    lora_alpha=32,
    lora_dropout=0.01,  # Slight dropout to prevent overfitting
    bias="none",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.01.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


In [19]:
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=UnslothVisionDataCollator(model, tokenizer),
    train_dataset=converted_dataset,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,  # Increased warmup
        max_steps=1000,  # More steps for better training, targeting SMAPE 20-40
        learning_rate=1e-4,  # Lower LR for stability
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",  # Cosine scheduler for better convergence
        seed=3407,
        output_dir="outputs",
        report_to="none",
        remove_unused_columns=False,
        dataset_text_field="",
        dataset_kwargs={"skip_prepare_dataset": True},
        max_length=2048,
    ),
)

trainer_stats = trainer.train()

Unsloth: Model does not have a default image size - using 512


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 8 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 82,169,856 of 3,836,792,832 (2.14% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,5.157500
2,5.783200
3,5.406100
4,5.617700
5,4.524200
6,4.000800
7,3.611500
8,3.256600
9,2.837700
10,2.672700


In [21]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
5.129 GB of memory reserved.


In [20]:
model.save_pretrained("/scientistbalaji/working/fine_tuned_model")
tokenizer.save_pretrained("/scientistbalaji/working/fine_tuned_model")
print("Model saved successfully.")

Model saved successfully.


In [22]:
FastVisionModel.for_inference(model)

def predict_price(catalog_content, image_link):
    try:
        image_filename = os.path.basename(image_link)
        image_path = f"/balaji/train/{image_filename}"
        if not os.path.exists(image_path):
            download_image(image_link, "/balaji/train/")
        image = Image.open(image_path).convert("RGB")

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"{instruction}\n\n{catalog_content}"},
                    {"type": "image", "image": image}
                ]
            }
        ]

        input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

        inputs = tokenizer(
            images=image,
            text=input_text,
            add_special_tokens=True,
            return_tensors="pt"
        ).to("cuda")

        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=True,  # Add sampling for diversity, temperature tuned
            temperature=0.8,  # Lower for more consistent predictions
            top_p=0.95  # Add top-p for better quality
        )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        price_str = generated_text.split("assistant")[-1].strip().replace("<|im_end|>", "").strip()
        price = abs(float(price_str))  # Ensure positive
        if price == 0:
            price = 0.01  # Fallback for zero
    except Exception as e:
        print(f"Error predicting for {image_link}: {e}")
        price = 0.01
    return price

In [23]:
os.makedirs("/balaji/train/", exist_ok=True)

download_images(sample_test['image_link'], '/balaji/train/')
sample_test['price'] = sample_test.apply(lambda row: predict_price(row['catalog_content'], row['image_link']), axis=1)
sample_test[['sample_id', 'price']].to_csv("/balaji/train/sample_test_out_pred.csv", index=False)
print("Predictions saved to sample_test_out_pred.csv")

100%|██████████| 100/100 [00:00<00:00, 206.19it/s]


Predictions saved to sample_test_out_pred.csv


In [25]:
import pandas as pd
import numpy as np

pred_df = pd.read_csv("/balaji/train/sample_test_out_pred.csv")
actual_df = pd.read_csv("/content/sample_test_out.csv")

merged_df = pred_df.merge(actual_df, on="sample_id", suffixes=('_pred', '_actual'))

def compute_smape(actual, predicted):
    n = len(actual)
    if n == 0:
        return np.nan
    absolute_error = np.abs(predicted - actual)
    denominator = (np.abs(actual) + np.abs(predicted)) / 2
    smape = np.mean(np.where(denominator != 0, absolute_error / denominator, 0)) * 100
    return smape

smape_score = compute_smape(merged_df['price_actual'], merged_df['price_pred'])
print(f"SMAPE Score on sample_test: {smape_score:.2f}%")

# If SMAPE not in 20-40, adjust training (e.g., more steps or data) and rerun Steps 9-13

SMAPE Score on sample_test: 105.39%
